<a href="https://colab.research.google.com/github/Thanesha2000/Book-recommender-system-/blob/main/trading_shit_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Install required libraries in Colab
!pip install yfinance pandas numpy gspread oauth2client scikit-learn
!pip install TA-Lib || echo "TA-Lib installation failed, using pandas for indicators"

import pandas as pd
import numpy as np
import yfinance as yf
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import logging
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime, timedelta
import time
import os

try:
    import talib
except ImportError:
    talib = None

# Configure logging
logging.basicConfig(
    filename='trading_bot.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
GOOGLE_SHEETS_CREDENTIALS = '/content/algo.json'  # Path to your Google Service Account JSON
SHEET_NAME = 'AlgoTradingBot'
STOCKS = ['RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS']  # NSE symbols
POSITION_SIZE = 100  # Shares per trade
BACKTEST_PERIOD = 180  # 6 months in days

def setup_google_sheets():
    """Set up Google Sheets client."""
    try:
        if not os.path.exists(GOOGLE_SHEETS_CREDENTIALS):
            logger.error(f"Google Sheets credentials file not found at {GOOGLE_SHEETS_CREDENTIALS}")
            raise FileNotFoundError(f"Google Sheets credentials file not found at {GOOGLE_SHEETS_CREDENTIALS}")

        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_SHEETS_CREDENTIALS, scope)
        client = gspread.authorize(creds)
        sheet = client.open(SHEET_NAME)
        logger.info("Connected to Google Sheets")
        return sheet
    except Exception as e:
        logger.error(f"Google Sheets setup failed: {e}")
        raise

def fetch_stock_data(symbol):
    """Fetch daily stock data using yfinance."""
    try:
        nse_symbol = symbol.replace('.BSE', '.NS')  # Map BSE to NSE if needed
        stock = yf.Ticker(nse_symbol)
        data = stock.history(period="6mo", interval="1d")
        if data.empty:
            logger.error(f"No data returned for {symbol} via yfinance.")
            return None
        data = data.rename(columns={
            'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'
        }).drop(columns=['Dividends', 'Stock Splits'])
        data['symbol'] = symbol
        logger.info(f"Fetched data for {symbol} via yfinance")
        return data
    except Exception as e:
        logger.error(f"Data fetch failed for {symbol}: {e}")
        return None

def calculate_indicators(data):
    """Calculate technical indicators if TA-Lib is available or as fallback."""
    if talib:
        data['rsi'] = talib.RSI(data['close'], timeperiod=14)
        data['sma20'] = talib.SMA(data['close'], timeperiod=20)
        data['sma50'] = talib.SMA(data['close'], timeperiod=50)
        data['macd'], data['macd_signal'], data['macd_hist'] = talib.MACD(data['close'])
    else:
        returns = data['close'].pct_change()
        pos_returns = returns.where(returns > 0, 0).rolling(window=14).mean()
        neg_returns = returns.where(returns < 0, 0).rolling(window=14).mean().abs()
        data['rsi'] = 100 - (100 / (1 + (pos_returns / (neg_returns + 1e-10))))
        data['sma20'] = data['close'].rolling(window=20).mean()
        data['sma50'] = data['close'].rolling(window=50).mean()
        data['macd'] = data['close'].ewm(span=12, adjust=False).mean() - data['close'].ewm(span=26, adjust=False).mean()
        data['macd_signal'] = data['macd'].ewm(span=9, adjust=False).mean()
        data['macd_hist'] = data['macd'] - data['macd_signal']
    return data.dropna()

def generate_signals(data):
    """Generate buy/sell signals based on RSI and MA crossover."""
    data = data.copy()  # Avoid SettingWithCopyWarning by working on a copy
    data.loc[:, 'signal'] = 0
    data.loc[(data['rsi'] < 40) & (data['sma20'] > data['sma50']), 'signal'] = 1
    data.loc[(data['rsi'] > 70) | (data['sma20'] < data['sma50']), 'signal'] = -1
    return data

def backtest_strategy(data):
    """Backtest the trading strategy and calculate P&L."""
    trades = []
    position = 0
    entry_price = 0
    total_pnl = 0
    wins = 0
    total_trades = 0

    for i in range(1, len(data)):
        if data['signal'].iloc[i] == 1 and position == 0:
            entry_price = data['close'].iloc[i]
            position = POSITION_SIZE
            trades.append({
                'date': data.index[i],
                'symbol': data['symbol'].iloc[i],
                'action': 'BUY',
                'price': entry_price,
                'pnl': 0,
                'ml_prediction': data.get('ml_prediction', 0).iloc[i]
            })
            logger.info(f"BUY {data['symbol'].iloc[i]} at {entry_price} on {data.index[i]}")
        elif data['signal'].iloc[i] == -1 and position > 0:
            exit_price = data['close'].iloc[i]
            pnl = (exit_price - entry_price) * POSITION_SIZE
            total_pnl += pnl
            total_trades += 1
            if pnl > 0:
                wins += 1
            trades.append({
                'date': data.index[i],
                'symbol': data['symbol'].iloc[i],
                'action': 'SELL',
                'price': exit_price,
                'pnl': pnl,
                'ml_prediction': data.get('ml_prediction', 0).iloc[i]
            })
            logger.info(f"SELL {data['symbol'].iloc[i]} at {exit_price} on {data.index[i]}, P&L: {pnl}")
            position = 0

    win_ratio = wins / total_trades if total_trades > 0 else 0
    return trades, total_pnl, win_ratio

def train_ml_model(data):
    """Train a Decision Tree model to predict next-day price movement."""
    data = data.copy()  # Avoid SettingWithCopyWarning by working on a copy
    try:
        features = ['rsi', 'sma20', 'sma50', 'macd', 'macd_signal', 'macd_hist', 'volume']
        data.loc[:, 'target'] = (data['close'].shift(-1) > data['close']).astype(int)
        X = data[features].dropna()
        y = data['target'].loc[X.index]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        model = DecisionTreeClassifier(max_depth=5, random_state=42)
        model.fit(X_train, y_train)

        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        f1 = f1_score(y_test, predictions)

        data.loc[:, 'ml_prediction'] = model.predict(data[features])
        logger.info(f"ML Model - Accuracy: {accuracy:.2f}, F1-Score: {f1:.2f}")
        return data, accuracy, f1
    except Exception as e:
        logger.error(f"ML model training failed: {e}")
        return data, 0, 0

def log_to_sheets(sheet, trades, summary_data, portfolio_data):
    """Log trades, summary P&L, and portfolio analytics to Google Sheets."""
    try:
        trade_ws = sheet.worksheet('Trade Log') if 'Trade Log' in [ws.title for ws in sheet.worksheets()] else sheet.add_worksheet('Trade Log', rows=1000, cols=10)
        # Convert numpy types to native Python types
        trade_data = [[t['date'].strftime('%Y-%m-%d'), t['symbol'], t['action'], float(t['price']), float(t['pnl']), int(t['ml_prediction'])] for t in trades]
        trade_ws.update(values=[['Date', 'Symbol', 'Action', 'Price', 'P&L', 'ML Prediction']] + trade_data, range_name='A1')

        summary_ws = sheet.worksheet('Summary P&L') if 'Summary P&L' in [ws.title for ws in sheet.worksheets()] else sheet.add_worksheet('Summary P&L', rows=100, cols=10)
        summary_data_converted = [[d['symbol'], float(d['total_pnl']), int(d['total_trades']), float(d['win_ratio'])] for d in summary_data]
        summary_ws.update(values=[['Symbol', 'Total P&L', 'Trades', 'Win Ratio']] + summary_data_converted, range_name='A1')

        portfolio_ws = sheet.worksheet('Portfolio Analytics') if 'Portfolio Analytics' in [ws.title for ws in sheet.worksheets()] else sheet.add_worksheet('Portfolio Analytics', rows=100, cols=10)
        portfolio_data_converted = [[k, float(v) if isinstance(v, (int, np.integer, np.floating)) else v] for k, v in portfolio_data.items()]
        portfolio_ws.update(values=[['Metric', 'Value']] + portfolio_data_converted, range_name='A1')

        logger.info("Data logged to Google Sheets")
    except Exception as e:
        logger.error(f"Google Sheets logging failed: {e}")

def main():
    """Main function to orchestrate the trading bot."""
    try:
        logger.info("Starting trading bot")
        sheet = setup_google_sheets()
        all_trades = []
        summary_data = []
        portfolio_pnl = 0
        portfolio_trades = 0
        portfolio_wins = 0

        for symbol in STOCKS:
            logger.info(f"Processing {symbol}")
            data = fetch_stock_data(symbol)
            if data is None:
                continue

            data = calculate_indicators(data)
            data = generate_signals(data)
            data, accuracy, f1 = train_ml_model(data)
            logger.info(f"ML Model for {symbol} - Accuracy: {accuracy:.2f}, F1-Score: {f1:.2f}")
            trades, total_pnl, win_ratio = backtest_strategy(data)
            all_trades.extend(trades)
            total_trades = len([t for t in trades if t['action'] == 'SELL'])
            summary_data.append({
                'symbol': symbol,
                'total_pnl': total_pnl,
                'total_trades': total_trades,
                'win_ratio': win_ratio
            })
            portfolio_pnl += total_pnl
            portfolio_trades += total_trades
            portfolio_wins += total_trades * win_ratio

        if not all_trades:
            logger.warning("No trades generated for any symbol.")

        portfolio_data = {
            'Total P&L': portfolio_pnl,
            'Total Trades': portfolio_trades,
            'Portfolio Win Ratio': portfolio_wins / portfolio_trades if portfolio_trades > 0 else 0
        }

        log_to_sheets(sheet, all_trades, summary_data, portfolio_data)
        logger.info("Trading bot completed successfully")
    except Exception as e:
        logger.error(f"Main function failed: {e}")

if __name__ == "__main__":
    main()

  Using cached ta_lib-0.6.4.tar.gz (381 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib)
TA-Lib installation failed, using pandas for indicators
